# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [57]:
import re
import os
import pandas as pd

In [58]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [61]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        char = re.sub(' |\*|<|>','_',char)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [62]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [63]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [64]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[58, 61, 77, 1, 58, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['_', 'b', 'r', '<UNK>', '_', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [65]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 30
BATCH_SIZE = 64
EMBED_DIM = 48
HIDDEN_SIZE = 8
MAX_LEN = 100
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)#TimeDistributed层的作用就是把Dense层应用到这10个具体的向量上，对每一个向量进行了一个Dense操作
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 100)
x_test shape: (3974, 100)
trainlabels shape: (6899, 100, 21)
testlabels shape: (3974, 100, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 48)           329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 16)           3648      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 21)           357       
_________________________________________________________________
crf_1 (CRF)                  (None, 100, 21)           945  

In [66]:
# save model
model.save("model/ch_ner_model.h5")

In [67]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体     
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        number += 1
        result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
        #result_words.append(('T'+str(number),tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取结尾的实体
    return result_words

In [68]:
maxlen = 100
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        #if len(sent_chars)<=maxlen:
        #    for k in range(len(sent_chars)):
        #        if sent_chars[k] == '，':
        #            y_ner_[k] = 'O'
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    ans = []
    for res in result_words:
        number = res[0]
        #tag_start_end = res[1]
        tag = res[1]
        start = res[2]
        end = res[3]
        word = res[4].replace(' ','_')
        ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end-1, word))
        #ans.append(, tag, start,end, "".join(word))
        print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start, end-1, word))
    result[i+1000] = ans

T1	DRUG_EFFICACY 138 141	清热解毒
T2	DRUG_EFFICACY 148 151	祛瘀止痛
T3	DRUG_EFFICACY 153 155	散结消
T4	SYMPTOM 167 170	小腹疼痛
T5	SYMPTOM 172 175	腰骶酸痛
T1	PERSON_GROUP 31 32	孕妇
T2	DISEASE 36 38	糖尿病
T3	DRUG_EFFICACY 76 77	补血
T4	DRUG_EFFICACY 79 80	活血
T5	SYMPTOM 87 90	月经量少
T6	SYMPTOM 95 100	血虚萎黄后错
T7	SYMPTOM 102 105	血虚萎黄
T8	SYMPTOM 107 110	风湿痹痛
T1	FOOD_GROUP 25 26	辛辣
T2	DISEASE 51 53	高血压
T3	DISEASE_GROUP 66 67	肾病
T4	SYMPTOM 90 93	月经紊乱
T5	SYMPTOM 109 110	眩晕
T6	PERSON_GROUP 192 193	儿童
T7	DRUG_DOSAGE 242 243	颗粒
T8	DRUG_TASTE 245 247	气微香
T9	DRUG_EFFICACY 255 258	滋养肝肾
T10	SYMPTOM 282 285	烘热汗出
T11	SYMPTOM 287 290	头晕耳鸣
T12	SYMPTOM 292 295	失眠多梦
T13	SYMPTOM 297 300	五心烦热
T14	SYMPTOM 302 305	腰背酸痛
T15	SYMPTOM 307 309	大便干
T16	SYMPTOM 312 315	心烦易怒
T17	SYMPTOM 317 319	舌红少
T18	SYNDROME 406 407	阴虚
T19	SYNDROME 408 409	肝旺
T20	DRUG_TASTE 487 489	气微香
T21	DRUG_TASTE 491 493	味微苦
T1	DISEASE 55 58	子宫肌瘤
T2	SYNDROME 59 62	气滞血瘀
T3	SYMPTOM 66 69	经期延长
T4	SYMPTOM 71 74	经量过多
T5	SYMPTOM 76 81	经色紫黯有块
T6	SYMPTOM 86 89	乳房胀痛
T7	DRUG_DOSA

T1	PERSON_GROUP 15 16	孕妇
T2	DISEASE 18 20	糖尿病
T3	DISEASE 35 36	感冒
T4	SYMPTOM 86 89	月经量少
T5	SYMPTOM 92 95	月经错后
T6	SYMPTOM 98 104	阴道不规则出血
T7	PERSON_GROUP 164 168	过敏体质者
T8	PERSON_GROUP 199 200	儿童
T9	DRUG_EFFICACY 284 287	补血活血
T10	DRUG_EFFICACY 289 292	调经止痛
T11	SYNDROME 296 297	血虚
T12	SYMPTOM 299 302	月经不调
T13	SYMPTOM 304 305	痛经
T14	PERSON_GROUP 306 307	儿童
T15	DRUG_DOSAGE 312 315	薄膜衣片
T16	DRUG_TASTE 332 333	气香
T17	DRUG_TASTE 335 336	味辛
T1	DRUG_DOSAGE 8 9	颗粒
T2	DRUG_TASTE 11 12	气香
T3	DRUG_TASTE 14 15	味甜
T4	DRUG_TASTE 17 17	辛
T5	DRUG_TASTE 19 21	微苦辛
T6	DRUG_TASTE 23 24	微苦
T7	SYMPTOM 65 68	贫血衰弱
T1	PERSON_GROUP 29 30	孕妇
T2	DRUG_EFFICACY 36 39	清热利湿
T3	SYNDROME 53 56	湿热下注
T4	SYMPTOM 58 61	白带过多
T5	DISEASE 63 65	盆腔炎
T6	DRUG_DOSAGE 85 87	胶囊剂
T7	DRUG_EFFICACY 134 137	清热解毒
T8	DRUG_INGREDIENT 171 172	川芎
T9	DRUG_INGREDIENT 174 176	鸡血藤
T10	DRUG_INGREDIENT 218 219	狗脊
T11	DRUG_EFFICACY 221 224	补益肝肾
T12	DRUG_EFFICACY 226 228	强腰膝
T13	DRUG_EFFICACY 230 232	除风湿
T14	DRUG_EFFICACY 234 236	利关节
T15	SYNDROME 279 28

T1	DRUG_INGREDIENT 41 42	藜芦
T2	DRUG_INGREDIENT 44 46	五灵脂
T3	DRUG_INGREDIENT 48 49	皂荚
T4	DISEASE 56 58	糖尿病
T5	SYMPTOM 71 74	月经提前
T6	DISEASE 81 82	感冒
T7	SYMPTOM 104 107	月经量少
T8	SYMPTOM 110 113	月经错后
T9	SYMPTOM 116 122	阴道不规则出血
T10	PERSON_GROUP 182 186	过敏体质者
T11	PERSON_GROUP 216 217	儿童
T12	DRUG_EFFICACY 254 257	补气养血
T13	DRUG_EFFICACY 259 262	祛瘀调经
T14	PERSON_GROUP 266 267	妇女
T15	SYNDROME 268 271	气血不足
T16	SYMPTOM 283 286	经行不畅
T17	PERSON_GROUP 311 312	孕妇
T18	DRUG_DOSAGE 346 347	稠膏
T19	DRUG_TASTE 349 350	味苦
T1	DRUG_EFFICACY 13 16	活瘀止血
T2	SYNDROME 20 23	脾肾两虚
T3	SYNDROME 25 28	气虚血瘀
T4	FOOD_GROUP 55 56	辛辣
T5	FOOD_GROUP 58 59	生冷
T6	DISEASE 68 69	感冒
T7	DISEASE 78 80	糖尿病
T8	SYMPTOM 114 117	月经过多
T9	SYMPTOM 120 126	阴道不规则出血
T10	PERSON_GROUP 168 170	过敏者
T11	PERSON_GROUP 174 178	过敏体质者
T12	PERSON_GROUP 213 214	儿童
T1	DRUG_EFFICACY 33 36	补血益气
T2	PERSON_GROUP 45 50	妇女月经量少
T3	SYMPTOM 58 59	痛经
T4	SYMPTOM 61 64	面色淡暗
T5	DRUG_DOSAGE 80 81	颗粒
T6	DRUG_TASTE 83 84	味苦
T7	DRUG_TASTE 86 86	涩
T8	DRUG_TASTE 88 89	微酸
T1	DR

T1	DRUG_DOSAGE 99 100	丸剂
T2	DRUG_DOSAGE 102 103	水丸
T3	FOOD_GROUP 113 114	生冷
T4	DISEASE 124 126	高血压
T5	DISEASE_GROUP 128 130	心脏病
T6	DISEASE_GROUP 139 140	肾病
T7	PERSON_GROUP 163 166	孕妇少女
T8	PERSON_GROUP 168 169	孕妇
T9	PERSON_GROUP 232 236	过敏体质者
T10	PERSON_GROUP 264 265	儿童
T11	DRUG_DOSAGE 363 364	水丸
T12	DRUG_TASTE 366 367	味苦
T13	DRUG_EFFICACY 371 372	清热
T14	DRUG_EFFICACY 374 375	除湿
T15	SYNDROME 382 385	湿热下注
T16	DISEASE_GROUP 389 391	带下病
T17	SYMPTOM 395 398	带下量多
T1	SYMPTOM 78 79	痛经
T2	SYMPTOM 81 82	闭经
T3	SYMPTOM 84 88	月经量过多
T1	DRUG_INGREDIENT 15 17	忍冬藤
T2	DRUG_INGREDIENT 19 21	蒲公英
T3	DRUG_EFFICACY 23 26	清热解毒
T4	DRUG_INGREDIENT 60 61	川芎
T5	DRUG_INGREDIENT 63 65	鸡血藤
T6	DRUG_INGREDIENT 107 108	狗脊
T7	DRUG_EFFICACY 110 113	补益肝肾
T8	DRUG_EFFICACY 115 117	强腰膝
T9	DRUG_EFFICACY 119 121	除风湿
T10	DRUG_EFFICACY 123 125	利关节
T11	SYNDROME 168 171	湿热下注
T12	SYMPTOM 173 176	白带过多
T13	DISEASE 178 180	盆腔炎
T14	DRUG_EFFICACY 207 210	清热利湿
T15	SYMPTOM 229 232	白带过多
T16	DISEASE 234 236	盆腔炎
T17	FOOD_GROUP 242 243	辛辣
T18

T1	PERSON_GROUP 36 37	孕妇
T2	DRUG_EFFICACY 72 75	祛瘀生新
T3	SYMPTOM 79 82	月经量少
T4	SYNDROME 158 161	气血两虚
T5	SYMPTOM 165 168	月经量少
T6	SYMPTOM 170 173	色淡质稀
T7	SYMPTOM 177 180	头晕心悸
T8	SYMPTOM 182 185	疲乏无力
T9	DISEASE 198 200	高血压
T10	DISEASE_GROUP 206 207	肾病
T11	DISEASE 209 211	糖尿病
T12	PERSON_GROUP 264 268	青春期少女
T13	PERSON_GROUP 270 274	更年期妇女
T14	SYMPTOM 297 300	阴道出血
T15	PERSON_GROUP 378 379	儿童
T16	DRUG_DOSAGE 442 443	颗粒
T17	DRUG_TASTE 445 446	味甜
T18	DRUG_TASTE 448 449	微苦
T1	PERSON_GROUP 64 65	孕妇
T2	DRUG_EFFICACY 84 87	温经活血
T3	SYNDROME 98 99	血瘀
T4	SYMPTOM 103 106	月经后期
T5	SYMPTOM 108 109	痛经
T6	SYMPTOM 113 116	行经后错
T7	SYMPTOM 120 123	小腹冷痛
T1	DRUG_EFFICACY 3 6	收缩子宫
T2	DRUG_EFFICACY 62 65	排出瘀血
T3	DRUG_EFFICACY 179 181	抗血栓
T4	DRUG_EFFICACY 295 297	抗贫血
T5	DRUG_EFFICACY 445 450	降低补体溶血
T6	DRUG_EFFICACY 456 460	改善微循环
T7	DRUG_EFFICACY 473 478	促进子宫修复
T8	SYMPTOM 557 560	恶露不行
T9	SYMPTOM 562 565	少腹疼痛
T10	SYMPTOM 579 582	阴道流血
T11	SYMPTOM 584 587	月经过多
T12	PERSON_GROUP 605 606	孕妇
T13	PERSON_GROUP 700 701	儿童
T14	P

...

T1	PERSON_GROUP 8 9	孕妇
T2	DRUG_EFFICACY 44 47	活血调经
T3	SYNDROME 51 52	血瘀
T4	SYMPTOM 56 59	月经不调
T5	DRUG_EFFICACY 86 89	祛瘀生新
T6	SYMPTOM 93 96	月经量少
T7	SYMPTOM 101 104	经来腹痛
T8	PERSON_GROUP 107 108	孕妇
T9	PERSON_GROUP 109 110	孕妇
T10	PERSON_GROUP 116 117	孕妇
T1	SYNDROME 20 23	气血两虚
T2	SYNDROME 26 27	血瘀
T3	SYMPTOM 38 43	月经周期错后
T4	SYMPTOM 45 48	行经量少
T5	SYMPTOM 49 54	月经周期错后
T6	SYMPTOM 56 59	行经量少
T7	SYMPTOM 61 64	精神不振
T8	SYMPTOM 66 69	肢体乏力
T9	DRUG_EFFICACY 70 73	益气养血
T10	DRUG_EFFICACY 75 78	活血调经
T11	SYNDROME 82 85	气血两虚
T12	SYNDROME 88 89	血瘀
T13	SYMPTOM 93 96	月经不调
T14	SYNDROME 149 152	气血两虚
T15	SYMPTOM 163 168	月经周期错后
T16	SYMPTOM 170 173	行经量少
T17	SYMPTOM 175 178	精神不振
T18	SYMPTOM 180 183	肢体乏力
T19	DRUG_DOSAGE 218 220	大蜜丸
T20	DRUG_EFFICACY 243 245	补气血
T1	PERSON_GROUP 14 15	孕妇
T2	SYNDROME 66 67	血瘀
T3	SYMPTOM 71 74	月经不调
T4	SYMPTOM 78 81	经水量少
T5	PERSON_GROUP 82 83	孕妇
T6	DRUG_EFFICACY 84 87	祛瘀生新
T7	SYMPTOM 91 94	月经量少
T8	SYMPTOM 99 102	经来腹痛
T9	PERSON_GROUP 138 139	孕妇
T1	SYNDROME 17 20	血虚气滞
T2	SYNDROME 22 25	下焦

T1	PERSON_GROUP 1 2	孕妇
T2	DRUG_EFFICACY 13 16	理气活血
T3	SYNDROME 23 26	气血两虚
T4	SYNDROME 28 29	气滞
T5	SYNDROME 30 31	血瘀
T6	SYMPTOM 35 38	月经不调
T7	SYMPTOM 42 45	月经提前
T8	SYMPTOM 47 50	月经错后
T9	SYMPTOM 52 55	月经量多
T10	SYMPTOM 57 60	神疲乏力
T11	SYMPTOM 62 65	行经腹痛
T12	DRUG_DOSAGE 77 79	水蜜丸
T1	DRUG_EFFICACY 1 2	养血
T2	SYMPTOM 12 15	月经不调
T3	SYMPTOM 17 20	经期腹痛
T4	DRUG_EFFICACY 71 76	养血调经止痛
T5	SYMPTOM 80 83	月经不调
T6	SYMPTOM 85 88	经期腹痛
T7	PERSON_GROUP 90 91	孕妇
T1	SYNDROME 22 25	气血两虚
T2	SYNDROME 46 49	气血两虚
T3	SYMPTOM 60 65	月经周期错后
T4	SYMPTOM 67 70	行经量少
T5	SYMPTOM 72 75	精神不振
T6	SYMPTOM 77 80	肢体乏力
T7	SYNDROME 102 105	气血两虚
T8	SYNDROME 108 109	血瘀
T9	SYMPTOM 120 125	月经周期错后
T10	SYMPTOM 127 130	行经量少
T11	SYMPTOM 132 135	精神不振
T12	SYMPTOM 137 140	肢体乏力
T13	DRUG_EFFICACY 217 219	补气血
T1	DRUG_EFFICACY 9 12	祛瘀生新
T2	SYMPTOM 16 19	月经量少
T3	PERSON_GROUP 105 106	孕妇
T4	PERSON_GROUP 178 179	孕妇
T5	SYNDROME 192 193	血瘀
T6	SYMPTOM 197 200	月经不调
T7	SYMPTOM 204 207	经水量少
T8	SYNDROME 208 209	血瘀
T9	PERSON_GROUP 211 212	孕妇
T1	PERSON_GROUP 34

T1	SYNDROME 10 13	气血两虚
T2	SYMPTOM 24 29	月经周期错后
T3	SYMPTOM 31 34	行经量少
T4	SYMPTOM 36 39	精神不振
T5	SYMPTOM 41 44	肢体乏力
T6	DRUG_EFFICACY 89 91	补气血
T7	SYNDROME 144 147	气血两虚
T8	SYNDROME 150 151	血瘀
T9	SYMPTOM 162 167	月经周期错后
T10	SYMPTOM 169 172	行经量少
T11	SYMPTOM 174 177	精神不振
T12	SYMPTOM 179 182	肢体乏力
T13	DRUG_EFFICACY 183 186	益气养血
T14	DRUG_EFFICACY 188 191	活血调经
T15	DRUG_EFFICACY 198 200	补气血
T1	PERSON_GROUP 14 15	孕妇
T2	DRUG_EFFICACY 34 37	活血调经
T3	SYNDROME 41 42	血瘀
T4	SYMPTOM 55 58	月经不调
T5	SYMPTOM 62 65	经水量少
T6	PERSON_GROUP 66 67	孕妇
T7	DRUG_EFFICACY 99 102	祛瘀生新
T8	SYMPTOM 106 109	月经量少
T9	PERSON_GROUP 212 213	孕妇
T1	PERSON_GROUP 55 56	孕妇
T2	PERSON_GROUP 61 62	孕妇
T3	SYMPTOM 93 99	子宫平滑肌收缩
T4	DRUG_INGREDIENT 117 118	艾叶
T5	DRUG_EFFICACY 121 124	抑制纤溶
T6	DRUG_EFFICACY 126 127	抗炎
T7	DRUG_EFFICACY 137 142	提高痛阈艾叶
T8	DRUG_EFFICACY 145 148	抑制纤溶
T9	DRUG_EFFICACY 150 151	抗炎
T10	DRUG_EFFICACY 166 167	镇痛
T11	SYMPTOM 179 182	子宫收缩
T12	SYMPTOM 231 234	月经不调
T13	SYMPTOM 236 237	痛经
T14	PERSON_GROUP 242 244	孕妇女
T15	SYMPTOM 

T1	SYNDROME 13 16	气血两虚
T2	SYNDROME 19 24	血瘀气血两虚
T3	SYNDROME 27 28	血瘀
T4	SYMPTOM 39 44	月经周期错后
T5	SYMPTOM 46 49	行经量少
T6	SYMPTOM 51 54	精神不振
T7	SYMPTOM 56 59	肢体乏力
T8	DRUG_EFFICACY 134 136	补气血
T9	SYNDROME 146 149	气血两虚
T10	SYMPTOM 160 165	月经周期错后
T11	SYMPTOM 167 170	行经量少
T12	SYMPTOM 171 176	月经周期错后
T13	SYMPTOM 178 181	行经量少
T14	SYMPTOM 183 186	精神不振
T15	SYMPTOM 188 191	肢体乏力
T1	SYNDROME 13 16	气血两虚
T2	SYNDROME 19 20	血瘀
T3	SYMPTOM 31 36	月经周期错后
T4	SYMPTOM 38 41	行经量少
T5	SYMPTOM 43 46	精神不振
T6	SYMPTOM 48 51	肢体乏力
T7	SYNDROME 52 55	气血两虚
T8	SYNDROME 58 59	血瘀
T9	SYMPTOM 63 66	月经不调
T10	DRUG_EFFICACY 67 70	益气养血
T11	SYNDROME 189 192	气血两虚
T12	SYMPTOM 203 208	月经周期错后
T13	SYMPTOM 210 213	行经量少
T14	SYMPTOM 215 218	精神不振
T15	SYMPTOM 220 223	肢体乏力
T16	DRUG_EFFICACY 226 228	补气血
T1	SYNDROME 69 72	下焦虚寒
T2	SYMPTOM 76 79	月经不调
T3	SYMPTOM 81 82	痛经
T4	PERSON_GROUP 120 121	孕妇
T5	PERSON_GROUP 127 130	孕妇胃痛
T6	SYMPTOM 137 138	尿频
T7	DRUG_EFFICACY 164 167	理气养血
T8	DRUG_EFFICACY 169 172	暖宫调经
T9	DRUG_EFFICACY 189 192	暖宫调经
T10	SYNDROME 

T1	DRUG_EFFICACY 1 4	通调气血
T2	DRUG_EFFICACY 6 9	止痛调经
T3	SYMPTOM 83 86	经期腹痛
T1	DRUG_EFFICACY 1 3	清湿热
T2	DRUG_EFFICACY 5 7	止带下
T3	SYNDROME 11 14	湿热下注
T4	SYMPTOM 16 19	赤白带下
T5	SYNDROME 41 44	湿热下注
T6	SYMPTOM 46 49	带下量多
T7	DRUG_EFFICACY 56 59	清热除湿
T1	DRUG_EFFICACY 1 4	通调气血
T2	DRUG_EFFICACY 6 9	止痛调经
T3	SYMPTOM 13 16	经期腹痛
T4	SYMPTOM 23 26	月经失调
T1	DRUG_EFFICACY 1 3	补气血
T2	SYNDROME 81 84	气血两虚
T3	SYNDROME 87 88	血瘀
T4	SYMPTOM 99 104	月经周期错后
T5	SYMPTOM 106 109	行经量少
T6	SYMPTOM 111 114	精神不振
T7	SYMPTOM 116 119	肢体乏力
T8	SYMPTOM 120 123	行经量少
T9	SYNDROME 211 214	气血两虚
T10	SYMPTOM 225 230	月经周期错后
T11	SYMPTOM 232 235	行经量少
T12	SYMPTOM 236 241	月经周期错后
T13	SYMPTOM 243 246	行经量少
T14	SYMPTOM 248 251	精神不振
T15	SYMPTOM 253 256	肢体乏力
T1	DRUG_EFFICACY 0 3	益气养血
T2	DRUG_EFFICACY 5 8	健脾补肾
T3	DRUG_EFFICACY 10 13	调经止带
T4	SYNDROME 17 20	脾肾两虚
T5	SYMPTOM 22 25	月经量少
T1	SYNDROME 17 20	气血两虚
T2	SYNDROME 23 24	血瘀
T3	SYMPTOM 35 40	月经周期错后
T4	SYMPTOM 42 45	行经量少
T5	SYMPTOM 47 50	精神不振
T6	SYMPTOM 52 55	肢体乏力
T7	SYNDROME 56 57	血瘀
T8	DRUG_EFFIC

In [ ]:
#“实体类别”、“起始位置”、“结束位置”以空格分隔

In [69]:
for i in range(1000,1500):
    with open('data/submit/%d.ann'%i,'w', encoding='utf-8') as wr:
        wr.write('\n'.join(result[i]))

In [ ]:
#for i in range(1000,1500):
#    pd.DataFrame(result[i]).to_csv('data/submit/%d.ann'%i,
#                                      sep='\t',
#                                      header = None,
#                                      index = 0,
#                                      encoding = 'utf-8')